In [3]:
from data_loader import ImgDataset,TgtImgDataset, VideoDataset
from sampler import RandomIdentityBatchSampler,RandomTrackletBatchSampler,RandomIdentitySampler
from data_manager import Mars,DukeMTMC

import torch
from torchvision import transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from model import FeatureExtractor,SrcReidModel,TgtPartModel,ModifiedTripletLoss,TripletLoss
from torch import optim
from torch.optim import lr_scheduler

In [55]:
def extract_feature(dataloader, feature_extractor):
    features = []
    pids = []
    camids = []
    tracklets = []
    paths = []
    with torch.no_grad():
        for batch_idx,batch in enumerate(dataloader):
            imgs = batch[0].cuda(3)
            pid = batch[1]
            camid = batch[2]
            path = list(batch[3])
            if len(batch) > 4:
                tracklet = batch[4]
                tracklets.append(tracklet)

            feature = feature_extractor(imgs)

            features += [feature[:,:,0].cpu()] 
            pids += [pid]
            camids += [camid]
            paths += path
            print(batch_idx)
        features = torch.cat(features,dim=0)
        pids = torch.cat(pids,dim=0).numpy()
        if len(tracklets) > 0:
            tracklets = torch.cat(tracklets,dim=0).numpy()
    return features,pids,camids,paths,tracklets
    

In [ ]:
def compute_distmat(a,b):
    m = a.size(0)
    n = b.size(0)
    distmat = torch.pow(a,2).sum(dim=1,keepdim=True).expand(m,n) + \
                         torch.pow(b,2).sum(dim=1,keepdim=True).expand(n,m).t()
    distmat.addmm_(1,-2,a,b.t())
    distmat = distmat.clamp(min=1e-12).sqrt() 
    return distmat.numpy()

In [ ]:
class CrossEntropyLabelSmooth(nn.Module):
    """Cross entropy loss with label smoothing regularizer.

    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.

    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """
    def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = Variable(targets, requires_grad=False)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss

In [13]:

class GlobalVar(object):
    def __init__(self):

        self._tgt_batch_size = 8
        self.seq_len = 4
        self.batch_size_sum = 64
        self.num_parts = 6
        self.gpu_ids = [3]
        self.feature_extractor = FeatureExtractor(self.num_parts)
        if len(self.gpu_ids > 0):
            self.use_gpu = True
            torch.cuda.set_device(self.gpu_ids)
            self.feature_extractor = nn.DataParallel(self.feature_extractor,self.gpu_ids)
        else:
            self.use_gpu = False

        self.save_path = './Logs'
        self.src_data = DukeMTMC()
        self.tgt_data = Mars()

        self.train_transfrom = T.Compose(
            [
                T.Resize((256,128)),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
            ]
        )

        self.test_transform = T.Compose(
            [
                T.Resize((256,128)),
                T.ToTensor(),
                T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
            ]
        )
        



    @property
    def tgt_batch_size(self):
        return self._tgt_batch_size

    @tgt_batch_size.setter
    def tgt_batch_size(self, value):

        if value*self.seq_len > self.batch_size_sum:
            self._tgt_batch_size = (
                self.batch_size_sum // self.seq_len)*self.seq_len
        elif value < 0:
            self._tgt_batch_size = 0
        else:
            self._tgt_batch_size = value

    @property
    def src_batch_size(self):
        return self.batch_size_sum - self._tgt_batch_size*self.seq_len

    def step(self):
        pass

    def set_global_var(self,name,value):
        setattr(self,name,value)

In [14]:
glob_var = GlobalVar()

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value.
       
       Code imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        

In [ ]:
def TrainStage1(object):
    def __init__(self,glob_var,opt):

        self.glob_var           = glob_var
        self.save_name          = '_stage1_'

        # 数据加载参数
        self.reid_train_data    = self.glob_var.src_data.train
        self.src_batchsize      = 64
        self.tgt_batchsize      = 64
        self.num_workers        = 4
        self.reid_sampler       = RandomIdentitySampler(glob_var.src_data.train,glob_var.seq_len)

        # 模型参数
        self.feature_extractor  = self.glob_var.feature_extractor
        self.num_classes        = len(set(self.reid_train_data['pid']))
        self.num_parts          = self.feature_extractor.num_parts

        # 优化器参数
        self.src_extract_lr     = 0.005
        self.src_model_lr       = 0.05
        self.src_weight_decay   = 5e-04
        self.src_step_size      = 40
        self.src_gamma          = 0.1
        self.tgt_extract_lr     = 0.005
        self.tgt_model_lr       = 0.05
        self.tgt_weight_decay   = 5e-04
        self.tgt_step_size      = 40
        self.tgt_gamma          = 0.1
        self.src_optim          = optim.Adam
        self.tgt_optim          = optim.Adam

        # Loss函数
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.triplet_loss       = TripletLoss()

        # 训练参数
        self.max_epoch          = 200
        self.use_triplet_loss   = True
        self.tgt_labels         = [torch.IntTensor([i]*self.tgt_batchsize) for i in range(self.num_parts)]

        if self.glob_var.use_gpu:
            for i in range(len(self.tgt_labels)):
                self.tgt_labels[i] = self.tgt_labels[i].cuda()
        
    def get_dataloader(self):

        self.src_loader = DataLoader(ImgDataset(self.reid_train_data,self.glob_var.train_transform),
                               batch_size = self.src_datasize,
                               shuffle = True,
                               num_workers = self.num_workers,
                               drop_last = True,
                               pin_memory = False,
                               sampler = self.reid_sampler)

        self.tgt_loader = DataLoader(VideoDataset(self.glob_var.tgt_data.train,self.glob_var.train_transform,self.glob_var.seq_len),
                               batch_size = self.tgt_batchsize//self.glob_var.seq_len,
                               shuffle = True,
                               num_workers = self.num_workers,
                               drop_last = True 
                               pin_memory = False)

    def get_model(self):

        self.src_model = SrcReidModel(self.num_classes)
        self.tgt_model = TgtPartModel(self.num_parts)

        if self.glob_var.use_gpu:
            self.src_model = nn.DataParallel(self.src_model,self.glob_var.gpu_ids)
            self.tgt_model = nn.DataParallel(self.tgt_model,self.glob_var.gpu_ids)
    
    def get_optimizer(self):

        self.src_optimizer = self.src_optim(
            [{'params':self.feature_extractor.parameters(),'lr':self.src_extract_lr},
            {'params':self.src_model.parameters(),'lr':self.src_model_lr}],
            weight_decay = self.src_weight_decay
        )
        
        self.tgt_optimizer = self.tgt_optim(
            [{'params':self.feature_extractor.parameters(),'lr':self.src_extract_lr},
            {'params':self.tgt_model.parameters(),'lr':self.tgt_model_lr}],
            weight_decay = self.tgt_weight_decay
        )

        self.src_lr_scheduler = lr_scheduler(self.src_optimizer,step_size=self.src_step_size,gamma=self.src_gamma)
        self.tgt_lr_scheduler = lr_scheduler(self.tgt_optimizer,step_size=self.tgt_step_size,gamma=self.tgt_gamma)

    def train_tgt_one_batch(self,batch):
        imgs = batch[0]
        if self.glob_var.use_gpu:
            imgs = imgs.cuda()
        features = self.feature_extractor(imgs)
        predicts = self.tgt_model(features)

        loss = self.cross_entropy_loss(predicts[0],self.tgt_labels[0])
        for i in range(len(predicts)-1):
            loss += self.cross_entropy_loss(predicts[i+1],self.tgt_labels[i+1])

        tgt_optimizer.zero_grad()
        loss.backward()
        tgt_optimizer.step()
        return loss
        
    def train_src_one_batch(self,batch):
        imgs = batch[0]
        labels = batch[1]
        if self.glob_var.use_gpu:
            imgs = imgs.cuda()
            labels = labels.cuda()
        features = self.feature_extractor(imgs)
        predicts = self.src_model(features)

        cross_loss = self.cross_entropy_loss(predicts[0],labels)
        for i in range(len(predicts) - 1):
            cross_loss += self.cross_entropy_loss(predicts[i+1],labels)
        loss = cross_loss

        if self.use_triplet_loss:
            tri_loss = self.triplet_loss(featuers[:,:,0],labels)
            loss += tri_loss
        src_optimizer.zero_grad()
        loss.backward()
        src_optimizer.step()
        return loss


    def _init_train(self):
        self.get_dataloader()
        self.get_model()
        self.get_optimizer()

    def train(self):
        self._init_train()
        for epoch in range(self.max_epoch):
            for src_batch in self.src_loader:
                src_loss = self.train_src_one_batch(src_batch)
                
            
            


In [1]:
!nvidia-smi

Sun Mar 29 08:13:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 78%   87C    P2   223W / 250W |   8365MiB / 11172MiB |     79%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 68%   83C    P2    83W / 250W |   8961MiB / 11172MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [4]:
tgt_labels = [torch.IntTensor([i]*64) for i in range(6)]

In [31]:
for i in range(len(tgt_labels)):
    if True:
        tgt_labels[i] = tgt_labels[i].cuda()
    tgt_labels[i][:,i] = 1

In [5]:
a = torch.tensor()

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32), tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=torch.int32), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=torch.int32), tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

In [ ]:
def TrainStage2(object):
    def __init__(self,glob_var):
        self.glob_var = glob_var
    def get_dataloader(self,)